### 01 Google Colab から OSS LLM への接続
- OPTION：VS Code から Google Colab の GPUサーバに接続
- LLMに接続（Hugging Faceのログインなし）
- LLMに接続（Hugging Faceのログイン<u>**あり**</u>）

**OPTION：VS Code から Google Colab の GPUサーバに接続する手順**
1. カーネルでcolabを選択
2. New Colab Serverを選択し、GPUを選択, GPUの種類（T4など）を選択
3. !nvidia-smiでGPUが選択されているか確認

In [1]:
# !nvidia-smiでGPUが選択されているか確認
# コマンドの「!」は、python ではなく、OS のコマンドという意味（シェルエスケープ）
!nvidia-smi

Thu Feb  5 11:05:08 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

**LLMに接続（Hugging Faceのログインなし）**
1. Google Colabに必要なライブラリをインストールする。
2. torchのGPUの使用可否、Hugging Faceのログイン状況を確認する。
3. transformersを使ってLLMモデルをHugging Faceから読み込みする。
4. モデルを読み込めているか確認する。確認後、一旦、モデルを削除。
5. transformersを使ってLLMモデルを<u>**量子化して**</u>読み込みする。
6. 読み込んだモデルのメモリ消費量が減っているか確認する。確認後、一旦、モデルを削除。

In [2]:
# Google Colabに必要なライブラリをインストールする。
# transformers accelerateはLLMの読み込み、bitsandbytesは量子化に必要
# -U は最新版をインストールするという指示
%pip install -U transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 19.2 MB/s eta 0:00:00:00:0100:01


In [3]:
# torchでcudaが使用可能か確認する。
import torch  # type: ignore

print(torch.cuda.is_available())

True


In [4]:
# Hugging Faceにはログインしていない状態です。
import os
from huggingface_hub import logout, whoami  # type: ignore

# もし既にHugging Faceにログインしている場合は、一旦ログアウトする
logout()  # トークンキャッシュを削除
if "HF_TOKEN" in os.environ:
    del os.environ["HF_TOKEN"]  # セッション内の環境変数も削除

try:
    user_info = whoami()
    print(f"ログイン成功: {user_info['name']} としてアクセス中")
except Exception:
    print("ログインしていません。")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(
Not logged in!


ログインしていません。


In [5]:
# transformersを使ってLLMモデルをHugging Faceから読み込みする。
# モデル読み込み後、モデルの大きさを出力させる。
import torch  # type: ignore
from transformers import AutoModelForCausalLM, AutoTokenizer  # type: ignore

model_name = "unsloth/gemma-3-1b-it"  # Hugging Faceのモデルの名称

# トークナイザ：テキストをLLMが読み込みできる形式に変換する
tokenizer = AutoTokenizer.from_pretrained(model_name)

# LLM モデルの読み込み
model = AutoModelForCausalLM.from_pretrained(
    model_name,  # Hugging Faceから読み込みするモデルの名称
    dtype=torch.bfloat16,  # 読み込みするモデルのデータ型
    device_map="auto",  # 読み込んだモデルを CPU / GPUに自動で割り当てする指示
)

# モデルのメモリ消費量（バイト）を取得し、GBに変換して表示
memory_footprint_bytes = model.get_memory_footprint()
memory_footprint_gb = memory_footprint_bytes / (1024 * 1024 * 1024)
print(f"モデルのメモリ消費量: {memory_footprint_gb:.2f} GB")

config.json:   0%|          | 0.00/902 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/340 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

モデルのメモリ消費量: 1.86 GB


In [6]:
# 読み込んだLLMの動作を確認する。
# 実際にLLMに質問し、会話を生成する。
from transformers import GenerationConfig, pipeline  # type: ignore

# LLMに生成させる文章の計算条件
# LM Studio（ローカルLLMを取り扱うデスクトップアプリ）のデフォルト値を流用
# LLMモデル内部にもgeneration_configが設定されており、設定しなくても動作するが、明示的に設定している。
generation_config = GenerationConfig(
    do_sample=True,  # Trueにしないと後段の設定が効かない。（Falseだとgreedy法）
    temperature=0.8,  # LLMが生成する文章のランダムさのパラメータ（値が大きい:多様、小さい:決定的）
    top_k=40,  # LLMが生成するランダムな文章の妥当性を向上させるパラメータ（値が小さい:候補が絞られる）
    top_p=0.9,  # LLMが生成するランダムな文章の妥当性を向上させるパラメータ（値が小さい:候補が絞られる）
    repetition_penalty=1.1,  # LLMが不要に繰り返し同じ出力をすることを抑制させるパラメータ（1.0=なし、値が大きい:抑制強い）
    max_new_tokens=512,  # LLMが生成する文章（トークン数）の上限（≠LM Studio）
)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

# LLMに入力するプロンプト
messages = [
    {"role": "user", "content": "日本で一番高い山は？日本語で回答して下さい。"},
]

# LLMにプロンプトを送信し、会話を生成
pipe(messages, generation_config=generation_config)

[{'generated_text': [{'role': 'user', 'content': '日本で一番高い山は？日本語で回答して下さい。'},
   {'role': 'assistant', 'content': '日本の最高峰である富士山です。\n'}]}]

In [7]:
# 読み込んだLLMを削除し、メモリを解放する。
import gc
import torch  # type: ignore

del model
gc.collect()
torch.cuda.empty_cache()

LLM モデルを**量子化**して読み込みさせる場合。**BitsAndBytesConfigはGPU動作が前提**のため、CPUで動作させると非常に遅いので注意。

In [8]:
# transformersを使ってLLMモデルを**量子化して**読み込みする。
# モデル読み込み後、モデルの大きさを出力させる。
# **BitsAndBytesConfigはGPU動作が前提**のため、CPUで動作させると非常に遅いので注意。
import torch  # type: ignore
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig  # type: ignore

model_name = "unsloth/gemma-3-1b-it"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4 ビットに量子化された形式で読み込むように指定
    bnb_4bit_quant_type="nf4",  # 4 ビット量子化のデータ型として NF4 を指定
    bnb_4bit_use_double_quant=True,  # 二重量子化の指定
    bnb_4bit_compute_dtype=torch.bfloat16,  # 計算時のデータ型を指定
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    dtype=torch.bfloat16,  # bnb_4bit_compute_dtypeと一致させる必要がある
    device_map="auto",
)

# モデルのメモリ消費量（バイト）を取得し、GBに変換して表示
memory_footprint_bytes = model.get_memory_footprint()
memory_footprint_gb = memory_footprint_bytes / (1024 * 1024 * 1024)
print(f"モデルのメモリ消費量: {memory_footprint_gb:.2f} GB")

Loading weights:   0%|          | 0/340 [00:00<?, ?it/s]

モデルのメモリ消費量: 0.89 GB


In [9]:
# **BitsAndBytesConfigはGPU動作が前提**のため、CPUで動作させると非常に遅いので注意。
# 読み込んだLLMの動作を確認する。
# 実際にLLMに質問し、会話を生成する。
from transformers import GenerationConfig, pipeline  # type: ignore

# LLMに生成させる文章の計算条件
# LM Studio（ローカルLLMを取り扱うデスクトップアプリ）のデフォルト値を流用
# LLMモデル内部にもgeneration_configが設定されており、設定しなくても動作するが、明示的に設定している。
generation_config = GenerationConfig(
    do_sample=True,  # Trueにしないと後段の設定が効かない。（Falseだとgreedy法）
    temperature=0.8,  # LLMが生成する文章のランダムさのパラメータ（値が大きい:多様、小さい:決定的）
    top_k=40,  # LLMが生成するランダムな文章の妥当性を向上させるパラメータ（値が小さい:候補が絞られる）
    top_p=0.9,  # LLMが生成するランダムな文章の妥当性を向上させるパラメータ（値が小さい:候補が絞られる）
    repetition_penalty=1.1,  # LLMが不要に繰り返し同じ出力をすることを抑制させるパラメータ（1.0=なし、値が大きい:抑制強い）
    max_new_tokens=512,  # LLMが生成する文章（トークン数）の上限（≠LM Studio）
)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

# LLMに入力するプロンプト
messages = [
    {"role": "user", "content": "日本で一番高い山は？日本語で回答して下さい。"},
]

# LLMにプロンプトを送信し、会話を生成
pipe(messages, generation_config=generation_config)

[{'generated_text': [{'role': 'user', 'content': '日本で一番高い山は？日本語で回答して下さい。'},
   {'role': 'assistant',
    'content': '日本の最高峰である富士山です。\n\n*   **標高:** 3,776.28m (12,389ft)\n*   **場所:** 長野県御洲町\n*   **特徴:** 日本第三の山であり、多くの登山者が訪れる人気の山です。\n\nより詳しい情報については、以下のサイトもご参照ください。\n\n*   **富士山登山 - Wikipedia:** [https://ja.wikipedia.org/wiki/富士山](https://ja.wikipedia.org/wiki/富士山)\n*   **富士山の高さと歴史 - 日本登山:** [https://www.j-mountain.jp/fujisan/](https://www.j-mountain.jp/fujisan/)'}]}]

In [10]:
# 読み込んだLLMを削除し、メモリを解放する。
import gc
import torch  # type: ignore

del model
gc.collect()
torch.cuda.empty_cache()

**LLMに接続（Hugging Faceのログイン<u>あり</u>）**
1. Google Colabに必要なライブラリをインストールする。torchのGPUの使用可否も確認。
2. Hugging FaceのAPIキーを設定する。
    - ケース1：Google Drive をマウントして、.envファイルから読み込み。（VS Code経由のため、Colab拡張機能の設定が必要）
    - ケース2：ダイアログを表示させ、直接APIキーを入力する。
3. transformersを使って<u>**認証が必要な**</u>LLMモデルをHugging Faceから読み込みする。
4. モデルを読み込めているか確認する。

In [11]:
# Google Colabに必要なライブラリをインストールする。
# transformers accelerateはLLMの読み込み、bitsandbytesは量子化に必要
# -U は最新版をインストールするという指示
%pip install -U transformers accelerate bitsandbytes

In [12]:
# torchでcudaが使用可能か確認する。
import torch  # type: ignore

print(torch.cuda.is_available())

True


**Hugging FaceのAPIキーを設定する。**
- ケース1：Google Drive をマウントして、.envファイルから読み込み。（VS Code経由のため、Colab拡張機能の設定が必要）
    > コード実行時は、webブラウザが開き、Google Drive接続の認証が必要になる。

In [13]:
# Google Driveのマウント（VS Code経由のため、Colab拡張機能の設定が必要）
# コード実行時は、webブラウザが開き、Google Drive接続の認証が必要
from google.colab import drive  # type: ignore

drive.mount("/content/drive")

Mounted at /content/drive


In [14]:
# ワーキングディレクトリをColabに認識させ、直下にあるファイルを確認
# .envファイルを確認することが目的。
import os
from dotenv import load_dotenv  # type: ignore

# ワーキングディレクトリを設定し、そのフォルダへ移動する
# ColabカーネルからVS Codeのワーキングディレクトリを読み取れないので、ハードコードしている
working_folder = "/content/drive/MyDrive/GEN_AI_RAG"
os.chdir(working_folder)

# 現在の場所を確認
print("現在の場所:", os.getcwd())

# ファイル一覧を表示
print("ファイル一覧:\n" + "\n".join(os.listdir()))

# .envファイル（環境変数が記載されているファイル）を読み込み
load_dotenv(".env")

現在の場所: /content/drive/MyDrive/GEN_AI_RAG
ファイル一覧:
.git
app
data
notebook
outputs
src
scripts
tests
.python-version
.venv
pyproject.toml
uv.lock
.vscode
.claude
.gitignore
CLAUDE.md
README.md
.env


True

In [15]:
# 環境変数（Hugging Faceのアクセストークン：HF_TOKEN）を読み込めているか確認
import os

# HF_TOKEN が存在するか確認
hf_token = os.environ.get("HF_TOKEN")

if hf_token:
    print(f"HF_TOKEN は登録されています。 (先頭数文字: {hf_token[:3]}...)")
else:
    print("HF_TOKEN は登録されていません。")


HF_TOKEN は登録されています。 (先頭数文字: hf_...)


**Hugging FaceのAPIキーを設定する。**
- ケース2：ダイアログを表示させ、直接APIキーを入力する。

In [16]:
# Hugging Faceのアクセストークンを読み込みする。
# 入力後、Google Colab内の環境変数として設定する。
import os
from getpass import getpass

# 環境変数に既にあればそれを使う。なければ入力を促す。
hf_token = os.getenv("HF_TOKEN") or getpass("Enter your Hugging Face token: ")
os.environ["HF_TOKEN"] = hf_token

if hf_token:
    print("トークンはセットされています。")
else:
    print("トークンはセットされていません。")

トークンはセットされています。


In [17]:
# Hugging Faceにログインする。
from huggingface_hub import login, whoami  # type: ignore

# Hugging Faceにログイン（要、Hugging Faceのアクセストークン）
login(hf_token)

# Hugging Faceにログインできているかを確認
try:
    user_info = whoami()
    print(f"ログイン成功: {user_info['name']} としてアクセス中")
except Exception:
    print("ログインしていません。")

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


ログイン成功: IS-reaction-wheel としてアクセス中


**transformersを使って認証が必要なLLMモデルをHugging Faceから読み込みする。**
- Googleのgemma-3-1b-itを使用します。事前にHugging Faceのサイトで、認証を受けることが必要です。

In [18]:
# transformersを使ってLLMモデルをHugging Faceから読み込みする。
# モデル読み込み後、モデルの大きさを出力させる。
import torch  # type: ignore
from transformers import AutoModelForCausalLM, AutoTokenizer  # type: ignore

model_name = "google/gemma-3-1b-it"  # Hugging Faceのモデルの名称

# トークナイザ：テキストをLLMが読み込みできる形式に変換する
tokenizer = AutoTokenizer.from_pretrained(model_name)

# LLM モデルの読み込み
model = AutoModelForCausalLM.from_pretrained(
    model_name,  # Hugging Faceから読み込みするモデルの名称
    dtype=torch.bfloat16,  # 読み込みするモデルのデータ型
    device_map="auto",  # 読み込んだモデルを CPU / GPUに自動で割り当てする指示
)

# モデルのメモリ消費量（バイト）を取得し、GBに変換して表示
memory_footprint_bytes = model.get_memory_footprint()
memory_footprint_gb = memory_footprint_bytes / (1024 * 1024 * 1024)
print(f"モデルのメモリ消費量: {memory_footprint_gb:.2f} GB")

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/340 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

モデルのメモリ消費量: 1.86 GB


In [19]:
# 読み込んだLLMの動作を確認する。
# 実際にLLMに質問し、会話を生成する。
from transformers import GenerationConfig, pipeline  # type: ignore

# LLMに生成させる文章の計算条件
# LM Studio（ローカルLLMを取り扱うデスクトップアプリ）のデフォルト値を流用
# LLMモデル内部にもgeneration_configが設定されており、設定しなくても動作するが、明示的に設定している。
generation_config = GenerationConfig(
    do_sample=True,  # Trueにしないと後段の設定が効かない。（Falseだとgreedy法）
    temperature=0.8,  # LLMが生成する文章のランダムさのパラメータ（値が大きい:多様、小さい:決定的）
    top_k=40,  # LLMが生成するランダムな文章の妥当性を向上させるパラメータ（値が小さい:候補が絞られる）
    top_p=0.9,  # LLMが生成するランダムな文章の妥当性を向上させるパラメータ（値が小さい:候補が絞られる）
    repetition_penalty=1.1,  # LLMが不要に繰り返し同じ出力をすることを抑制させるパラメータ（1.0=なし、値が大きい:抑制強い）
    max_new_tokens=512,  # LLMが生成する文章（トークン数）の上限（≠LM Studio）
)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

# LLMに入力するプロンプト
messages = [
    {"role": "user", "content": "日本で一番高い山は？日本語で回答して下さい。"},
]

# LLMにプロンプトを送信し、会話を生成
pipe(messages, generation_config=generation_config)

[{'generated_text': [{'role': 'user', 'content': '日本で一番高い山は？日本語で回答して下さい。'},
   {'role': 'assistant', 'content': '富士山です。'}]}]